In [ ]:
import os
import keras
!pip3 install keras_nlp
import keras_nlp
import tensorflow as tf
from tensorflow import keras


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.5/337.5 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 96.0 MB/s eta 0:00:00


In [ ]:
# Data
BATCH_SIZE = 64
SEQ_LEN = 128
MIN_TRAINING_SEQ_LEN = 450

# Model
EMBED_DIM = 256
FEED_FORWARD_DIM = 256
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 5000  # Limits parameters in model.

# Training
EPOCHS = 6

# Inference
NUM_TOKENS_TO_GENERATE = 80


In [ ]:
keras.utils.get_file(
    origin="https://dldata-public.s3.us-east-2.amazonaws.com/simplebooks.zip",
    extract=True,
)
dir = os.path.expanduser("~/.keras/datasets/simplebooks/")

# Load simplebooks-92 train set and filter out short lines.
raw_train_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/train.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

# Load simplebooks-92 validation set and filter out short lines.
raw_val_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/valid.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
)



282386239/282386239 [==============================] - 10s 0us/step


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
# Train tokenizer vocabulary
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)


In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True,
)


In [ ]:
# packer adds a start token
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)


def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels


# Tokenize and split into train and label sequences.
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)


In [ ]:
inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)
# Embedding.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)
x = embedding_layer(inputs)
# Transformer decoders.
for _ in range(NUM_LAYERS):
    decoder_layer = keras_nlp.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x)  # Giving one argument only skips cross-attention.
# Output.
outputs = keras.layers.Dense(VOCAB_SIZE)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer="adam", loss=loss_fn, metrics=[perplexity])


In [ ]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        1312768   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 dense (Dense)               (None, None, 5000)        128500

In [ ]:
model.fit(train_ds, validation_data=val_ds, verbose=2, epochs=EPOCHS)


Epoch 1/6
3169/3169 - 341s - loss: 4.5635 - perplexity: 96.2858 - val_loss: 4.1107 - val_perplexity: 61.6287 - 341s/epoch - 107ms/step
Epoch 2/6
3169/3169 - 230s - loss: 4.0483 - perplexity: 57.5224 - val_loss: 3.9932 - val_perplexity: 54.6582 - 230s/epoch - 73ms/step
Epoch 3/6
3169/3169 - 229s - loss: 3.9335 - perplexity: 51.2811 - val_loss: 3.9180 - val_perplexity: 50.7189 - 229s/epoch - 72ms/step
Epoch 4/6
3169/3169 - 229s - loss: 3.8706 - perplexity: 48.1511 - val_loss: 3.8804 - val_perplexity: 48.8415 - 229s/epoch - 72ms/step
Epoch 5/6
3169/3169 - 230s - loss: 3.8283 - perplexity: 46.1560 - val_loss: 3.8545 - val_perplexity: 47.5279 - 230s/epoch - 73ms/step
Epoch 6/6
3169/3169 - 230s - loss: 3.7953 - perplexity: 44.6590 - val_loss: 3.8097 - val_perplexity: 45.5479 - 230s/epoch - 73ms/step


In [ ]:

prompt_tokens = tf.convert_to_tensor([tokenizer.token_to_id("[BOS]")])

In [ ]:
def token_logits_fn(inputs):
    cur_len = inputs.shape[1]
    output = model(inputs)
    return output[:, cur_len - 1, :]  # return next token logits


In [ ]:
output_tokens = keras_nlp.utils.greedy_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Greedy search generated text: \n{txt}\n")


Greedy search generated text: 
b'[BOS] " i \' m going to be a good deal of trouble , " said the old man . " i \' m going to be a good man , and i \' m going to be a good man . i \' m going to be a good man , and i \' m going to be a good man . i \' m going to be a good man , and i \' m going to be'



In [ ]:
output_tokens = keras_nlp.utils.beam_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    num_beams=10,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Beam search generated text: \n{txt}\n")


Beam search generated text: 
b'[BOS] " i don \' t know what i \' m going to do . i don \' t know what i \' m going to do . i don \' t know what i \' m going to do . i don \' t know what i \' m going to do . i don \' t know what i \' m going to do . i don \' t know what i \' m going to do .'



In [ ]:
output_tokens = keras_nlp.utils.random_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Random search generated text: \n{txt}\n")


Random search generated text: 
b'[BOS] " we may flemington india devils , though indeed , had this trouble with the children , " announced annie ; " or " them , kneeled in high regards the sea in her mind of england , and ornament except your loath - operation may have been steadily decorated in war . i am fearfully received from a concurmur'



In [ ]:
output_tokens = keras_nlp.utils.top_k_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    k=10,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-K search generated text: \n{txt}\n")


Top-K search generated text: 
b'[BOS] in the morning the boys were dressed in the morning as they were riding at the side of the wagon , and there was no time to ride back at night , and it was not a very much better to take the rider . when the girls got behind their coats and walked to the wagon , and were all ready and waiting . they had the best of them , and were to ride over ,'



In [ ]:
output_tokens = keras_nlp.utils.top_p_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    p=0.5,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search generated text: \n{txt}\n")


Top-P search generated text: 
b'[BOS] she had done all the same , and she had to be married . she had taken a long project of the house , and she was in a new place . she had never seen such a person . she had not only got a dollar , but the more she liked to be married . it was only a small , well - worn place , and her heart was glad to see'



In [ ]:
class TopKTextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model using top-k."""

    def __init__(self, k):
        self.k = k

    def on_epoch_end(self, epoch, logs=None):
        output_tokens = keras_nlp.utils.top_k_search(
            token_logits_fn,
            prompt_tokens,
            max_length=NUM_TOKENS_TO_GENERATE,
            k=self.k,
            from_logits=True,
        )
        txt = tokenizer.detokenize(output_tokens)
        print(f"Top-K search generated text: \n{txt}\n")


text_generation_callback = TopKTextGenerator(k=10)
# Dummy training loop to demonstrate callback.
model.fit(train_ds.take(1), verbose=2, epochs=2, callbacks=[text_generation_callback])


Epoch 1/2
Top-K search generated text: 
b'[BOS] when they had taken the little party to the camp on the bouquet and the confederate captain . it was late now , indeed . when it was completed i was in command of the party . the girls were all in the neighbourhood , and i could see that there was no doubt , for i was not at all the company , and in the morning i saw'

1/1 - 9s - loss: 3.8761 - perplexity: 48.3112 - 9s/epoch - 9s/step
Epoch 2/2
Top-K search generated text: 
b'[BOS] " i am a little too , " the doctor said . " it was a good deal like you , but i am not sure , and you see how much of it . i don \' t think i have any of you or any more . but the man who will tell me about you now has to tell him about it ; the man , who is a man in the country of the city'

1/1 - 9s - loss: 3.7646 - perplexity: 43.4369 - 9s/epoch - 9s/step


In [ ]:

text_generation_callback = TopKTextGenerator(k=10)
# Dummy training loop to demonstrate callback.
model.fit(train_ds.take(10), verbose=2, epochs=2, callbacks=[text_generation_callback])

Epoch 1/2
Top-K search generated text: 
b"[BOS] he was a tall and nobleman with his hand , with an old and noble youthful hand , and in the midst of his great adversal in the old man ' s eyes , and he could not have known his brother ' s name as his son as a son of a man , but had he , and the chief had a strong , well , was so indul"

10/10 - 10s - loss: 3.7872 - perplexity: 44.3147 - 10s/epoch - 986ms/step
Epoch 2/2
Top-K search generated text: 
b'[BOS] " it \' s the best , " continued mr . dave . " they were to be a peck of water , and they made their way in the direction of the river . they were all in the river bank at a spot where their boats was going to the north , but we got on a short distance , and i made a little shady creek in two days , and'

10/10 - 10s - loss: 3.7339 - perplexity: 41.9981 - 10s/epoch - 961ms/step


In [ ]:
class TopKTextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model using top-k."""

    def __init__(self, k):
        self.k = k

    def on_epoch_end(self, epoch, logs=None):
        output_tokens = keras_nlp.utils.top_k_search(
            token_logits_fn,
            prompt_tokens,
            max_length=NUM_TOKENS_TO_GENERATE,
            k=self.k,
            from_logits=True,
        )
        txt = tokenizer.detokenize(output_tokens)
        print(f"Top-K search generated text: \n{txt}\n")


text_generation_callback = TopKTextGenerator(k=10)
# Dummy training loop to demonstrate callback.
for i in range(10):
  model.fit(train_ds.take(1), verbose=2, epochs=2, callbacks=[text_generation_callback])



Epoch 1/2
Top-K search generated text: 
b'[BOS] " the shrewd man was the first to be a big , and was quite sure , " said the girl . " i have not been so happy as to be , and i am glad to know it . " and there were no great things on the landlord of his uncle ; and the little old man , who had been a good man . he had not seen the world'

1/1 - 14s - loss: 3.4981 - perplexity: 33.1378 - 14s/epoch - 14s/step
Epoch 2/2
Top-K search generated text: 
b'[BOS] the two lads were not to be afraid . the latter were to see that the scientists were the same , that a subsequently of civilization , the scotch and compound . the boys in a position in which he had been in the colonial , and , in his present century , were the colonialist generals'

1/1 - 9s - loss: 3.5452 - perplexity: 34.6899 - 9s/epoch - 9s/step
Epoch 1/2
Top-K search generated text: 
b'[BOS] he now went up , thinking he would try , but he was not going to see if he was going to be sure that it would be hard if he could find out wha

In [ ]:
class TopKTextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model using top-k."""

    def __init__(self, k):
        self.k = k

    def on_epoch_end(self, epoch, logs=None):
        output_tokens = keras_nlp.utils.top_k_search(
            token_logits_fn,
            prompt_tokens,
            max_length=NUM_TOKENS_TO_GENERATE,
            k=self.k,
            from_logits=True,
        )
        txt = tokenizer.detokenize(output_tokens)
        print(f"Top-K search generated text: \n{txt}\n")


text_generation_callback = TopKTextGenerator(k=10)
# Dummy training loop to demonstrate callback.
for i in range(10):
  model.fit(val_ds.take(1), verbose=2, epochs=2, callbacks=[text_generation_callback])

Epoch 1/2
Top-K search generated text: 
b'[BOS] the boys were to wait for some time , and then they went back to the place where they had left . it was the same , and the other two boys were not to be disturbed by the rest , who did not know that their father was not to blame for the doctor . they could have heard the others say they could not understand that they were too weak and helpless . they were too young'

1/1 - 4s - loss: 3.8351 - perplexity: 46.7020 - 4s/epoch - 4s/step
Epoch 2/2
Top-K search generated text: 
b'[BOS] " it is a great time to have the right of this , and a hundred times , " said he . " i have been there many days to come here to - day , and you know that it is a little too good for us to be out . we \' re glad to think about the whole affair . it is just the right thing you can do . it was a little better off'

1/1 - 4s - loss: 3.7596 - perplexity: 43.3168 - 4s/epoch - 4s/step
Epoch 1/2
Top-K search generated text: 
b'[BOS] when the day was over she found the b